In [1]:
import numpy as np
from flow.multiagent_envs import MultiWaveAttenuationMergePOEnv
from flow.scenarios import MergeScenario
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from ray.tune import run_experiments

import gym, ray
from ray.rllib.agents.ddpg import DDPGAgent, DEFAULT_CONFIG
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

benchmark_name = 'merge0'
benchmark_name = 'multi_merge'

In [2]:
alg_run = "DDPG"
num_rollouts = 1
num_cpus = 2
gae_lambda = 0.97
step_size = 5e-4

In [3]:
ray.init(num_cpus=num_cpus, include_webui=False, ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-20_02-08-57_2053/logs.
Waiting for redis server at 127.0.0.1:46813 to respond...
Waiting for redis server at 127.0.0.1:56996 to respond...
Starting the Plasma object store with 13.355121049 GB memory using /dev/shm.


{'node_ip_address': '169.237.32.118',
 'object_store_addresses': ['/tmp/ray/session_2019-04-20_02-08-57_2053/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-04-20_02-08-57_2053/sockets/raylet'],
 'redis_address': '169.237.32.118:46813',
 'webui_url': ''}

In [8]:
benchmark = __import__(
    "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.flow_params
horizon = flow_params['env'].horizon
sim_step = flow_params['sim'].sim_step

def make_env(create_env):
    def _thunk():
        env = create_env()
        return env
    return _thunk

create_env, env_name = make_create_env(params=flow_params, version=0)

In [9]:
# Register as rllib env
register_env(env_name, create_env)

In [10]:
config = DEFAULT_CONFIG.copy()

In [11]:
config["num_workers"] = min(num_cpus, num_rollouts)
config["train_batch_size"] = horizon * num_rollouts
config["horizon"] = horizon
config['timesteps_per_iteration'] = int(horizon / sim_step)
config['clip_actions'] = False  # FIXME(ev) temporary ray bug
config["model"]["fcnet_hiddens"] = [100, 50, 25]
config["observation_filter"] = "NoFilter"

# save the flow params for replay
flow_json = json.dumps(
    flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
config['env_config']['flow_params'] = flow_json
config['env_config']['run'] = alg_run

In [12]:
exp_tag = {
    "run": alg_run,
    "env": env_name,
    "config": {
        **config
    },
    "checkpoint_freq": 25,
    "max_failures": 999,
    "stop": {
        "training_iteration": 3
    },
    "num_samples": 1,
}

In [13]:
trials = run_experiments({
        flow_params["exp_tag"]: exp_tag
    })

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs
Memory usage on this node: 4.4/33.4 GB

Created LogSyncer for /headless/ray_results/multi_merge/DDPG_MultiWaveAttenuationMergePOEnv-v0_0_2019-04-20_02-10-29tklxb4m0 -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 2/2 CPUs, 0/1 GPUs
Memory usage on this node: 4.4/33.4 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - DDPG_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING

Result for DDPG_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-20_02-11-11
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 4a68d09a9de641cba562acd145c43cf7
  hostname: kronos
  info:
    grad_time_ms: .nan
    learner: {}
    max_exploration: 0.1
    min_exploration: 0.1
    num_steps_sampled: 1502
    num_steps_trained: 0
    num_target_